In [26]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams["figure.figsize"] = (12, 8) # (w, h)

In [27]:
def sim_leverage(proxy, leverage=1, expense_ratio = 0.0, initial_value=1.0):
    """
    Simulates a leverage ETF given its proxy, leverage, and expense ratio.
    
    Daily percent change is calculated by taking the daily percent change of
    the proxy, subtracting the daily expense ratio, then multiplying by the leverage.
    """
    val = proxy['Close']
    pct_change = (val - val.shift(1)) / val.shift(1)
    pct_change = (pct_change - expense_ratio / 252) * leverage
    sim = (1 + pct_change).cumprod() * initial_value
    sim[0] = initial_value
    return sim


In [28]:
def process_yahoo_csv(file_name):
    df = pd.read_csv(file_name, 
                     parse_dates=True,
                     index_col=0)

    price_ratio = df['Adj Close']/df['Close']
    for column in ["Open", "High", "Low", "Close"]:
        df[column] = df[column]*price_ratio

    return df[["Open", "High", "Low", "Close", "Volume"]]


In [29]:
vfinx_df = process_yahoo_csv("VFINX.csv")
vustx_df = process_yahoo_csv("VUSTX.csv")
nasdaq_df = process_yahoo_csv("NASDAQ.csv")
# nasdaq_df['Close'] /= 100

upro_sim_df = vfinx_df.copy()
tmf_sim_df = vustx_df.copy()
tqqq_sim_df = nasdaq_df.copy()

upro_sim_df['Close'] = sim_leverage(upro_sim_df, leverage=3.0, expense_ratio=0.02)
tmf_sim_df['Close'] = sim_leverage(tmf_sim_df, leverage=3.0, expense_ratio=0.02)
tqqq_sim_df['Close'] = sim_leverage(tqqq_sim_df, leverage=3.0, expense_ratio=0.02)

for column in ["Open", "High", "Low"]:
    upro_sim_df[column] = upro_sim_df["Close"]
    tmf_sim_df[column] = tmf_sim_df["Close"]
    tqqq_sim_df[column] = tqqq_sim_df["Close"]
upro_sim_df["Volume"] = 0
tmf_sim_df["Volume"] = 0
tqqq_sim_df["Volume"] = 0


In [30]:
# resample to month
upro_sim_df = upro_sim_df.groupby(pd.Grouper(freq="M")).last()
tmf_sim_df = tmf_sim_df.groupby(pd.Grouper(freq="M")).last()
tqqq_sim_df = tqqq_sim_df.groupby(pd.Grouper(freq="M")).last()
vfinx_df = vfinx_df.groupby(pd.Grouper(freq="M")).last()
vustx_df = vustx_df.groupby(pd.Grouper(freq="M")).last()
nasdaq_df = nasdaq_df.groupby(pd.Grouper(freq="M")).last()


In [31]:
upro_sim_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
1980-01-31,1.253497,1.253497,1.253497,1.253497,0
1980-02-29,1.255083,1.255083,1.255083,1.255083,0
1980-03-31,0.904496,0.904496,0.904496,0.904496,0
1980-04-30,1.014009,1.014009,1.014009,1.014009,0
1980-05-31,1.180158,1.180158,1.180158,1.180158,0


In [32]:
start = datetime.datetime(1986, 5, 19)
end = datetime.datetime(2020, 4, 20)

upro_sim = bt.feeds.PandasData(dataname=upro_sim_df, fromdate=start, todate=end)
tmf_sim = bt.feeds.PandasData(dataname=tmf_sim_df, fromdate=start, todate=end)
vfinx = bt.feeds.PandasData(dataname=vfinx_df, fromdate=start, todate=end)
tqqq_sim = bt.feeds.PandasData(dataname=tqqq_sim_df, fromdate=start, todate=end)


In [33]:
class BuyAndHold(bt.Strategy):
    def next(self):
        if not self.getposition(self.data).size:
            self.order_target_percent(self.data, target=1.0)


In [34]:
def backtest(datas, strategy, plot=False, **kwargs):
    cerebro = bt.Cerebro()
    for data in datas:
        cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns, timeframe=bt.TimeFrame.Months)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addstrategy(strategy, **kwargs)
    results = cerebro.run()
    if plot:
        cerebro.plot()
    return (results[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
            results[0].analyzers.returns.get_analysis()['rnorm100'],
            results[0].analyzers.sharperatio.get_analysis()['sharperatio'])


In [35]:
dd, cagr, sharpe = backtest([vfinx], BuyAndHold, plot=True)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 50.64%
CAGR: 9.15%
Sharpe: 0.605


In [36]:
dd, cagr, sharpe = backtest([upro_sim], BuyAndHold)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 96.72%
CAGR: 9.94%
Sharpe: 0.460


In [37]:
dd, cagr, sharpe = backtest([tmf_sim], BuyAndHold)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 49.87%
CAGR: 14.75%
Sharpe: 0.554


In [38]:
class AssetAllocation(bt.Strategy):
    params = (
        ('asset_alloc', None),
    )
    def __init__(self):
        self.UPRO = self.datas[0]
        self.TMF = self.datas[1]
        self.TQQQ = self.datas[2]
        self.counter = 0
        
    def next(self):
        if self.params.asset_alloc is not None:
            self.order_target_percent(self.UPRO, target=self.params.asset_alloc[0])
            self.order_target_percent(self.TQQQ, target=self.params.asset_alloc[2])
            self.order_target_percent(self.TMF, target=self.params.asset_alloc[1])


In [39]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True, asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 55.47%
CAGR: 17.73%
Sharpe: 0.643


In [40]:
sharpes = {}
for perc_equity in range(0, 101, 5):
    ratio_equity = perc_equity/100.0
    asset_alloc=[ratio_equity*1.0, 1-ratio_equity, ratio_equity*0.0]
    sharpes[perc_equity] = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, asset_alloc=asset_alloc)[2]


In [41]:
series = pd.Series(sharpes)
ax = series.plot(title="UPRO/TMF allocation vs Sharpe")
ax.set_ylabel("Sharpe Ratio")
ax.set_xlabel("Percent Portfolio UPRO");
print(f"Max Sharpe of {series.max():.3f} at {series.idxmax()}% equity")


<IPython.core.display.Javascript object>

Max Sharpe of 0.721 at 35% equity


In [42]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True,  asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 55.47%
CAGR: 17.73%
Sharpe: 0.643


In [43]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True,  asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 55.47%
CAGR: 17.73%
Sharpe: 0.643


In [44]:
bt_result = []
for start_year in range(1987, 2020):
    for end_year in range(start_year+2, 2021):

        start = datetime.datetime(start_year, 1, 1)
        end = datetime.datetime(end_year, 1, 1)

        upro_sim = bt.feeds.PandasData(dataname=upro_sim_df, fromdate=start, todate=end)
        tmf_sim = bt.feeds.PandasData(dataname=tmf_sim_df, fromdate=start, todate=end)
        vfinx = bt.feeds.PandasData(dataname=vfinx_df, fromdate=start, todate=end)
        vustx = bt.feeds.PandasData(dataname=vustx_df, fromdate=start, todate=end)
        tqqq_sim = bt.feeds.PandasData(dataname=tqqq_sim_df, fromdate=start, todate=end)
        for pct_equity in range(0, 101, 20):
            ratio_equity = pct_equity/100.0
            # asset_alloc=[ratio_equity*0.6, 1-ratio_equity, ratio_equity*0.4]
            # dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, asset_alloc=asset_alloc)
            asset_alloc=[ratio_equity, 1-ratio_equity, 0]
            dd, cagr, sharpe = backtest([vfinx, vustx, vustx], AssetAllocation, asset_alloc=asset_alloc)
            bt_result.append({'start':start, 'end': end, 'cagr': cagr, 
                              'dd':dd, 'sharpe':sharpe, 'pct_equity':pct_equity})
            print(f"Start {start_year}, End {end_year}, %eq %{pct_equity}, Max Drawdown: {dd:.2f}, CAGR: {cagr:.2f}, Sharpe: {sharpe:.3f}")
bt_result = pd.DataFrame(bt_result)
bt_result.to_csv('bench_result_monthly.csv')

IndexError: list index out of range

In [11]:
bt_result = pd.read_csv('bt_result_monthly.csv', parse_dates=True, index_col=0)
bt_result['start'] = pd.to_datetime(bt_result['start'],format='%Y-%m-%d')
bt_result['end'] = pd.to_datetime(bt_result['end'],format='%Y-%m-%d')

In [12]:
bt_result.tail()

,start,end,cagr,dd,sharpe,pct_equity
5803,2018-01-01,2020-01-01,21.669767,25.945466,0.674474,60
5804,2018-01-01,2020-01-01,21.647537,29.724026,0.640853,70
5805,2018-01-01,2020-01-01,21.447458,33.811381,0.612811,80
5806,2018-01-01,2020-01-01,21.495047,37.884303,0.592945,90
5807,2018-01-01,2020-01-01,21.249372,42.747155,0.573758,100


In [13]:
bt_result['horizon'] = (bt_result['end'].dt.year - bt_result['start'].dt.year)
bt_horizon = bt_result[(bt_result['horizon'] % 5 ==0)]
horizon5 = bt_result[(bt_result['horizon'] == 5)]

In [16]:
bt_horizon.sort_values('dd').drop_duplicates(['start', 'horizon'],keep='first')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEAB7EEE0>,
      dtype=object)

In [17]:
bt_horizon.sort_values('cagr').drop_duplicates(['start', 'horizon'],keep='last')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEB40BDC0>,
      dtype=object)

In [18]:
bt_horizon.sort_values('sharpe').drop_duplicates(['start', 'horizon'],keep='last')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEC5FE520>,
      dtype=object)

In [20]:
bt_horizon[(bt_horizon['pct_equity']==40)]['dd'].hist(by=bt_horizon['horizon'], bins=range(8,53,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFED225400>,
      dtype=object)

In [22]:
bt_horizon[(bt_horizon['pct_equity']==40)]['cagr'].hist(by=bt_horizon['horizon'], bins=range(-2,43,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEE278610>,
      dtype=object)

In [23]:
bt_horizon[(bt_horizon['pct_equity']==100)]['cagr'].hist(by=bt_horizon['horizon'])#, bins=range(-2,43,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEF568C10>,
      dtype=object)

In [24]:
bt_horizon[(bt_horizon['pct_equity']==100)]['dd'].hist(by=bt_horizon['horizon'])#, bins=range(8,53,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFED2BBD30>,
      dtype=object)